# Utilities

## PushBackCharStream

In [ ]:
#export
from collections.abc import Iterator

class PushBackCharStream:
    def __init__(self, chars):
        self.iterator = chars if isinstance(chars, Iterator) else iter(chars)
        self.pushed_back = []
        self.line = 0
        self.col = 0
        self.line_history = [0]
        self.__reached_end = False

    def __iter__(self):
        return self

    def __next__(self):
        if self.__reached_end:
            raise StopIteration
            
        if self.pushed_back:
            char = self.pushed_back.pop()
        else:
            try:
                char = next(self.iterator)
            except StopIteration:
                self.__reached_end = True
                return None

        # advanced character position
        if char == '\n':
            self.line += 1
            self.col = 0
        else:
            self.col += 1

        # Save last line history for pushback
        if self.line >= len(self.line_history):
            self.line_history.append(0)
        self.line_history[self.line] = self.col

        return char

    def push_back_single(self, char: str):
        if char is None:
            return
        
        self.pushed_back.append(char)

        # Reverse the position
        if self.col == 0:
            self.line -= 1
            self.col = self.line_history[self.line]
        else:
            self.col -= 1
        
        # Reset EOF since we know we are not there anymore
        self.__reached_end = False
            
    def push_back(self, chars: str):
        if chars is None:
            self.__reached_end = False
            return
        
        for c in reversed(chars):
            self.push_back_single(c)

    @property
    def empty(self):
        return len(self.pushed_back) == 0 and self.__reached_end
    
    def starting_line_col_info(self):
        return (self.line, self.col - 1)
    
    def ending_line_col_info(self):
        return (self.line, self.col)

In [ ]:
# PushBackCharStream
# tests

stream = PushBackCharStream('123456789')
assert list(stream) == list(iter('123456789')) + [None], "Pushback should mimic string Iterator with None at end"

stream = PushBackCharStream('123')
assert next(stream) == '1', "Pushback should return chars in order"
assert next(stream) == '2', "Pushback should return chars in order"
assert next(stream) == '3', "Pushback should return chars in order"

stream = PushBackCharStream('12\n3')
assert stream.col == 0 and stream.line == 0,                   "Keeps track of column and line position"
assert next(stream) and stream.col == 1 and stream.line == 0, "Keeps track of column and line position"
assert next(stream) and stream.col == 2 and stream.line == 0, "Keeps track of column and line position"
assert next(stream) and stream.col == 0 and stream.line == 1, "Keeps track of column and line position"

stream = PushBackCharStream('12\n3')
assert next(stream) == '1' and stream.col == 1 and stream.line == 0
stream.push_back('1')
assert next(stream) == '1' and stream.col == 1 and stream.line == 0,"Can push back characters on to same line"

stream = PushBackCharStream('12\n3')
next(stream);next(stream);next(stream)
assert next(stream) == '3' and stream.col == 1 and stream.line == 1, "Can pushback characters spanning lines"
stream.push_back('3'); 
assert stream.col == 0 and stream.line == 1,                         "Can pushback characters spanning lines"
stream.push_back('\n')
assert stream.col == 2 and stream.line == 0,                         "Can pushback characters spanning lines"

stream = PushBackCharStream('12')
next(stream)
assert stream.starting_line_col_info() == (0, 0), 'Starting line col info is previous character position'
assert stream.ending_line_col_info() == (0, 1),   'Ending line col info is next character position'

In [ ]:
#export
from collections import namedtuple

LineColInfo = namedtuple('LineColInfo', ['start_line', 'start_col', 'end_line', 'end_col'])